# Importing Packages

In [2]:
import json
import os
import random
import matplotlib.pyplot as plt
import cv2

# --------------------------------------------------------------------------------------

### Creating annotations for each image - food class and polygon points - normalized

In [11]:
with open("../data/external/original_id_to_new_id.json", "r") as file:
    original_id_to_new_id = json.load(file)
original_id_to_new_id["1565"]

{'name': 'bread-wholemeal', 'new_id': 0}

In [24]:
def mapping_image_id_to_polygon_points(annotations_json_file_path: str):
    """
    This function return a dict, a key is an image id, and the value
    is the food classes and polygon points(normalized) of different classes present in the image.
    """
    with open(annotations_json_file_path, "r") as file:
        annotations_json_file = json.load(file)
    
    dct = {}
    for annotation in annotations_json_file["annotations"]:
        image_id = annotation["image_id"]
        food_class_original_id = annotation["category_id"]
        food_class_new_id = original_id_to_new_id[str(food_class_original_id)]["new_id"]
        segmentation = annotation["segmentation"]
        if image_id not in dct:
            dct[image_id] = []

        w = h = 0
        for img in annotations_json_file["images"]:
            if img["id"] == image_id:
                w = img["width"]
                h = img["height"]
                break
        
        for seg in segmentation:
            for i in range(len(seg)):
                if i%2==0:
                    seg[i] = seg[i]/w
                else:
                    seg[i] = seg[i]/h
            seg.insert(0, food_class_new_id)
            dct[image_id].append(seg)
    return dct

In [14]:
# Test on Val data
dct = mapping_image_id_to_polygon_points("../data/external/public_validation_set_2.0/annotations.json")

In [25]:
def doJob(data_path):
    """
    This function creats a text file for each image in the dataset and put
    the annotations(normalized format) inside this text
    """
    annotations_json_file_path = data_path+"/annotations.json"
    image_id_to_annotations = mapping_image_id_to_polygon_points(annotations_json_file_path)
    for image in os.listdir(data_path+"/images"):
        image_name = os.path.splitext(os.path.basename(data_path+"images/"+image))[0]
        image_id = int(image_name)
        
        annotation_txt_file_path = data_path+"/annotations-segmentation-normalized/"+image_name+".txt"
        with open(annotation_txt_file_path, "w") as file:
            for annotation in image_id_to_annotations[image_id]:
                file.write(" ".join(map(str, annotation)) + "\n")  # Write annotations

In [27]:
# Create annotations for the images in public_validation_set_2
doJob("../data/external/public_validation_set_2.0")

# Create annotations for the images in public_training_set_release_2.0
doJob("../data/external/public_training_set_release_2.0")